In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [4]:
path = '/Users/liamanderson/Desktop/Instacart Basket Analysis'

In [5]:
df_all_high_activity = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_all_high_activity.pkl'))

In [6]:
# 7.
## aggregate each of the data profiles
## First we will get rid of the outliers within the prices column
## check to see how many outlers there are under prices

df_all_high_activity[df_all_high_activity['prices'] >100].shape[0]

4877

In [7]:
## get rid of all of the outliers

df_cleaned = df_all_high_activity[df_all_high_activity['prices'] <= 100]

In [8]:
df_cleaned.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order',
       'reordered', 'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'loyalty_flag', 'spending_flag',
       'order_frequency_flag', 'First Name', 'Surname', 'Gender', 'State',
       'Age', 'date_joined', 'n_dependants', 'marital_status', 'income',
       '_merge', 'region', 'max_order', 'exclusion_flag', 'age_group',
       'income_group', 'dependants_flag'],
      dtype='object')

In [9]:
## We can use days since prior order for usage frequency but for expenditure we will have to create a new column with the price of the entire order

total_orders_subset = df_cleaned.groupby(['user_id', 'age_group', 'income_group', 'dependants_flag', 'loyalty_flag', 'spending_flag', 'order_frequency_flag','region', 'order_number', 'days_since_prior_order'])['prices'].sum().reset_index()

In [10]:
total_orders_subset.head()

,user_id,age_group,income_group,dependants_flag,loyalty_flag,spending_flag,order_frequency_flag,region,order_number,days_since_prior_order,prices
0,1,young_adult,low_income,has dependants,New customer,Low spender,Non-frequent customer,south,2,15.0,35.4
1,1,young_adult,low_income,has dependants,New customer,Low spender,Non-frequent customer,south,3,21.0,28.2
2,1,young_adult,low_income,has dependants,New customer,Low spender,Non-frequent customer,south,4,29.0,26.0
3,1,young_adult,low_income,has dependants,New customer,Low spender,Non-frequent customer,south,5,28.0,57.0
4,1,young_adult,low_income,has dependants,New customer,Low spender,Non-frequent customer,south,6,19.0,25.0


In [11]:
## change the name of the prices column to total orders

total_orders_subset.rename(columns = {'prices' : 'order_total'}, inplace = True)

In [12]:
total_orders_subset.columns

Index(['user_id', 'age_group', 'income_group', 'dependants_flag',
       'loyalty_flag', 'spending_flag', 'order_frequency_flag', 'region',
       'order_number', 'days_since_prior_order', 'order_total'],
      dtype='object')

In [13]:
## aggregate for age_group

df_cleaned.groupby('age_group').agg({'days_since_prior_order': ['mean', 'min', 'max']})

days_since_prior_order           
                              mean  min   max
age_group                                    
middle_aged              10.819525  0.0  30.0
senior                   10.800550  0.0  30.0
young_adult              10.811175  0.0  30.0

In [14]:
total_orders_subset.groupby('age_group').agg({'order_total': ['mean', 'min', 'max']})

order_total             
                   mean  min     max
age_group                           
middle_aged   78.442839  1.0  1092.2
senior        78.449148  1.0   875.0
young_adult   79.091596  1.0   842.1

In [15]:
## aggregate for income_group

df_cleaned.groupby('income_group').agg({'days_since_prior_order': ['mean', 'min', 'max']})

days_since_prior_order           
                                mean  min   max
income_group                                   
high_income                10.769337  0.0  30.0
low_income                 11.175808  0.0  30.0
middle_income              10.762138  0.0  30.0

In [16]:
total_orders_subset.groupby('income_group').agg({'order_total': ['mean', 'min', 'max']})

order_total             
                     mean  min     max
income_group                          
high_income     83.757277  1.0  1092.2
low_income      52.673028  1.0   875.0
middle_income   81.929747  1.0   852.5

In [17]:
## aggregate for number of dependants

df_cleaned.groupby('dependants_flag').agg({'days_since_prior_order': ['mean', 'min', 'max']})

days_since_prior_order           
                                  mean  min   max
dependants_flag                                  
has dependants               10.816265  0.0  30.0
no dependants                10.791879  0.0  30.0

In [18]:
total_orders_subset.groupby('dependants_flag').agg({'order_total': ['mean', 'min', 'max']})

order_total             
                       mean  min     max
dependants_flag                         
has dependants    78.728823  1.0  1092.2
no dependants     78.482647  1.0   875.0

In [19]:
# 8.
## We already have a column with data regarding region but we need to import and merge the departments dataset with the df_cleaned data set
## First import department data set

df_department = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'))

In [20]:
df_department.head()

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [21]:
## rename column to department_id

df_department.rename(columns={'Unnamed: 0':'department_id'},inplace=True)

In [22]:
df_department.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [23]:
df_cleaned.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,n_dependants,marital_status,income,_merge,region,max_order,exclusion_flag,age_group,income_group,dependants_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,1,married,49620,both,midwest,32,include,senior,low_income,has dependants
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,married,49620,both,midwest,32,include,senior,low_income,has dependants
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,2,married,158302,both,northeast,5,include,senior,high_income,has dependants
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,...,2,married,57797,both,west,26,include,middle_aged,middle_income,has dependants
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,...,3,living with parents and siblings,34171,both,south,9,include,young_adult,low_income,has dependants


In [24]:
## check data types

df_department.dtypes

department_id     int64
department       object
dtype: object

In [25]:
df_cleaned['department_id'].dtypes

dtype('int64')

In [26]:
df_cleaned['price_range_loc'].head()

0    Mid-range product
1    Mid-range product
2    Mid-range product
5    Mid-range product
6    Mid-range product
Name: price_range_loc, dtype: object

In [27]:
## data set is to large for me to be able to merge so we will get rid of some unecessary columns

df_cleaned.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order',
       'reordered', 'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'loyalty_flag', 'spending_flag',
       'order_frequency_flag', 'First Name', 'Surname', 'Gender', 'State',
       'Age', 'date_joined', 'n_dependants', 'marital_status', 'income',
       '_merge', 'region', 'max_order', 'exclusion_flag', 'age_group',
       'income_group', 'dependants_flag'],
      dtype='object')

In [28]:
df = df_cleaned.drop(['aisle_id', 'add_to_cart_order','First Name','Surname', '_merge', 'max_order'], axis=1)

In [29]:
df.to_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'df.4.10(part1 Q8).pkl'))

In [30]:
df_department.to_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'df_departments.pkl'))

In [31]:
total_orders_subset.to_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'total_orders_subset.pkl'))